# <font color='#eb3483'> Transforming Pandas DataFrames and Series </font>

Where pandas really shines is in it's ability to transform data - allowing you to tidy up a dataset in just a few lines of code. In this notebook, we'll be exploring this functionality. Remember, most of a data scientist's time is spent cleaning and transforming data!

In this notebook we will cover:

1. Removing rows and columns
1. Mathematical operations
1. Naming operations
1. Aggregations

We're going to work with the AirBnB dataset again, but only a few columns. When we load a dataframe from the csv file, we can specify the columns we want to use with `usecols`.

In [1]:
import pandas as pd
columns = ["room_id", "host_id", "room_type", "neighborhood", "reviews", "overall_satisfaction","accommodates", "bedrooms", "price"]
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
df.head()

FileNotFoundError: ignored

## <font color='#eb3483'> 1. Removing Rows and Columns </font>
To remove rows and columns, we can use dataframe's `.drop` method. By default `.drop` removes rows based on the row name (**not** row position).

Drop has two important arguments:
* `index/columns`: Specify the index values and/or column names that you want to drop.
* `inplace`: With this argument, you can chose if you want to transform the original dataframe or if you want the drop function to return a copy of the transformed dataframe. The default value is ```False```.

To find out more on the ```.drop``` method and its arguments, type ```?df.drop```.

In [ ]:
df.head()

In [ ]:
df.drop(index=[6499, 17031], inplace=True) 
# drops the rows where room_id is 6499 or 17031 (this is the index)
# and returns nothing (the original dataframe was changed, but not returned)

In [ ]:
df.head()

In [ ]:
df.drop(columns=["reviews", "price"]) 
# drops two columns
# and returns the modified data frame (a new df was created, the original df was not changed)

You can drop rows and columns at the same time. e.g. 

```python
df.drop(index = [6499, 17031], columns=["reviews", "price"])
```

Columns can also be dropped using Python's ```del``` function:
```python
del df['reviews']
```
Note that this is an "inplace" operation, so it modifies the original dataframe, but does not return anything. It can only be applied to one column at a time and only when the column name is provided inside square brackets.

This will work:
```python
del df['reviews'], df['price']
```

These will **not** work:
```python
del df[['reviews','price']]
del df.reviews
```

## <font color='#eb3483'> 2. Mathematical Operations </font>

Very often we want to manipulate a column to get something that makes more sense. Maybe getting a value per hour, or price per week or turning a birthdate into an age etc. 

### <font color='#eb3483'>  Multiplication </font>

You can either use the `*` or the `.multiply()` method to multiply two columns or multiply columns by a number.

For example, we can calculate the weekly price for the listings:

In [ ]:
df["price_per_week"] = df["price"] * 7 # or df["price_per_week"] = df.price.multiply(7) 
df.head()

### <font color='#eb3483'>   Division </font>
We can use either `/` or the `.divide()` to divide.

For example, we can calculate the number of people per bedroom:

In [ ]:
df["people_per_bedroom"] = df.accommodates / df.bedrooms # or df.accommodates.divide(df.bedrooms)
df.head()

<font color='#eb3483'> Exercise: </font> Try adding or subtracting columns.

In [ ]:
df.reviews + df.accommodates

## <font color='#eb3483'> 3. Naming Operations </font>

We can change the name of the columns by changing the column names list `df.columns`. For example, we can rename the columns and make them capitalized.

In [ ]:
new_column_names = df.columns.str.title() # converts the current column names to title case
df.columns = new_column_names
df.head()

Alternatively, we can use the ```.rename``` method to rename either the row indices (with ```index=...```) or column names (with ```columns=...```). In both cases, we provide a dictionary that maps from the old name (dict key) to the new name (dict value). For example:

In [ ]:
df.rename(columns = {"Host_Id": "My_Funky_New_Name", 
                   "Room_Type": "Something_Else"
                  }) # could use inplace=True to modify original

## <font color='#eb3483'> 4. Aggregations </font>
The goal of aggregations is to allow us to get an aggregated view of sub-sections of our data. Before we begin, let's read in our data again to erase all the edits that we may have made above:

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns)
df.head()

### <font color='#eb3483'>  Groupby </font>

`groupby` allows us to group the dataframe based on its features.

More precisely, Pandas' ```groupby``` enables a process called [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html).
* **split**: Separates the dataframe based on the specified groups
* **apply**: Applies a function to each one of the groups
* **combine**: Combines the results into a new dataframe

For example, if we want to know how many listings there are in each neighborhood. This means that for every ```neighborhood``` (the "group"), we want to count up the unique values of ```room_id```. 

We start by grouping the data:

In [ ]:
df.groupby("host_id")

This returns a `DataFrameGroupBy ` object, which is a special dataframe object that separates the dataframe by group.

In [ ]:
df.nunique() # gives you the number of unique values in each column

In [ ]:
(df.groupby("neighborhood")).nunique() # gives you the number of unique values in each column BY group

In [ ]:
# just for the "room_id" column:
(df.groupby("neighborhood")).room_id.nunique()

By default, the columns we use to group become the index, if we want them to stay as columns we can use the argument `as_index = False`.

In [ ]:
df.groupby("neighborhood", as_index=False).room_id.nunique()

<font color='#eb3483'> Exercise: </font> Calculate the average listing price by room type and host. **Hint:** Use the ```.mean()``` method to get the average (in place of ```.nunique()``` above).

In [ ]:
df.groupby(["room_type", "host_id"], as_index=False).price.mean()

We can apply any function to a grouped dataframe and pandas will pass the function on to the underlying split dataframes under the hood:

In [ ]:
df.groupby(["neighborhood"]).price.describe()

### <font color='#eb3483'>  Aggregate </font>

We have seen how to apply in-built methods like ```.sum``` and ```.mean``` to groups in a dataframe. What if we want to apply some other funky functions to each group, possibly ones that we have written ourselves? The pandas `aggregate` method allows us to do just that!

In [ ]:
def funky(x):
    some_field = (min(x)+max(x))/2
    return some_field # this returns a single number

In [ ]:
df.groupby(["neighborhood"]).price.aggregate(funky)

In [ ]:
df.groupby(["neighborhood"]).price.aggregate(  [min, funky, max]   ) # multiple functions all at once!

The `aggregate` function even let's us specify what columns we want to apply each aggregation function to using a dictionary:

In [ ]:
df.groupby(["neighborhood"]).aggregate({'price': [funky, max], 'accommodates':min})

### <font color='#eb3483'>  Pivot Table </font>

The `.pivot_table` method performs the same function as pivot tables in Excel. It turns rows into columns based on the values on the columns (that is, it "pivots" the data).

This function has different arguments:

- `index`: the columns whose values should become rows
- `columns`: the columns whose values should become columns
- `values`: the columns we want to aggregate
- `aggfunc`: the aggregate function applied to the values (mean by default)

For example, if we want to calculate the average satisfaction by ```room_type``` for each ```neighborhood```:

In [ ]:
df = pd.read_csv("data/airbnb.csv", usecols=columns, index_col="room_id").sort_index()
df.pivot_table(index="neighborhood", 
               columns='room_type', 
               values='overall_satisfaction',
               aggfunc='mean').head()

<hr>

# <font color='#eb3483'> LET'S PRACTICE! </font>

# <font color='#eb3483'> Transforming Pandas DataFrames and Series  </font>

Work on these excercises for 10 mins (or for homework depending on how well we do with time)




![image.png](attachment:image.png)

For these exercises we are going to use a new dataset, the 2016 US Primary elections (`primary_results.csv` in our data folder). Start by importing pandas and reading in our data:

In [1]:
import pandas as pd
votes = pd.read_csv("C:\Current_Work\primary_results.csv")


In [2]:
votes.head()

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182
1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800
2,Alabama,AL,Baldwin,1003.0,Democrat,Bernie Sanders,2694,0.329
3,Alabama,AL,Baldwin,1003.0,Democrat,Hillary Clinton,5290,0.647
4,Alabama,AL,Barbour,1005.0,Democrat,Bernie Sanders,222,0.078


The dataset has the following columns:

- *state*
- *state_abbreviation* 
- *county* 
- *fips* county identifier
- *party* 
- *candidate* 
- *votes* votes the candidate got in the county
- *fraction_votes* percentage of the total county votes the candidate got

For each problem - think about how you would work this out first. Talk yourself through each step (or even jot it down) and then code it.

### <font color='#eb3483'> Exercise 1 </font>
Overall, which percentage of votes did every party get?

In [3]:
total_votes = votes.votes.sum()
votes.groupby("party")["votes"].sum() / total_votes

party
Democrat      0.487331
Republican    0.512669
Name: votes, dtype: float64

### <font color='#eb3483'> Exercise 2 </font>

Who is the democrat candidate that got the most votes in manhattan? and in the state of New York?

In [4]:
votes[(votes.county=="Manhattan")&(votes.party=="Democrat")].sort_values(by="votes", ascending=False).head(1)

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
15012,New York,NY,Manhattan,36061.0,Democrat,Hillary Clinton,177496,0.663


In [5]:
votes[(votes.state_abbreviation=="NY")&(votes.party=="Democrat")].groupby(
    "candidate"
)[["votes"]].sum().reset_index().sort_values(by="votes", ascending=False).head(1)

,candidate,votes
1,Hillary Clinton,1054083


### <font color='#eb3483'> Exercise 3 </font>
How many votes did Donald Trump receive in Texas?

In [6]:
votes[(votes.candidate=="Donald Trump")&(votes.state_abbreviation=="TX")].votes.sum()

757618

### <font color='#eb3483'> Exercise 4 </font>

Let's consider democrat states those where the democrats got more votes and republican states those where the republican candidates got more votes. Which states are democrat and which republican?


*hint: one way to find out is by doing a pivot table using the sum as an aggregating function*

In [7]:
votes_by_party = pd.pivot_table(votes, values=["votes"], index="state",
                                   columns="party", aggfunc="sum")["votes"].reset_index()

democrat_states = votes_by_party[votes_by_party.Democrat> votes_by_party.Republican].state.unique()
republican_states = votes_by_party[votes_by_party.Democrat < votes_by_party.Republican].state.unique()

In [11]:
votes_by_party

party,state,Democrat,Republican
0,Alabama,386327.0,837632.0
1,Alaska,539.0,21930.0
2,Arizona,399097.0,435103.0
3,Arkansas,209448.0,396523.0
4,California,3442623.0,1495574.0
5,Colorado,121184.0,NaN
6,Connecticut,322485.0,208817.0
7,Delaware,92609.0,67807.0
8,Florida,1664003.0,2276926.0
9,Georgia,757340.0,1275601.0


In [8]:
democrat_states

array(['California', 'Connecticut', 'Delaware', 'Hawaii', 'Illinois',
       'Kentucky', 'Louisiana', 'Maryland', 'Massachusetts', 'New Jersey',
       'New Mexico', 'New York', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'Vermont', 'West Virginia'], dtype=object)

In [9]:
republican_states

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'Florida', 'Georgia',
       'Idaho', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'North Carolina', 'Ohio', 'Oklahoma', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia',
       'Washington', 'Wisconsin', 'Wyoming'], dtype=object)

### <font color='#eb3483'> Exercise 5 </font>

In how many of the republican states was Donald Trump the most voted republican candidate?

In [19]:
## give me only rows where the party is Republican
republican_votes = votes[votes.party=="Republican"]


## give me only rows where the states column contain republican_states and then grouby state and candidate
## the sum the votes and sort the values based on states and votes (by states in ascending order and by votes in descending order))
## and then groupby state,make the index false and then get the top 1 row
candidate_breakdown_republican = (republican_votes[republican_votes.state.isin(republican_states)]
 .groupby(["state", "candidate"], as_index=False)["votes"]
 .sum()
.sort_values(by=["state", "votes"], ascending=[True, False])
.groupby("state", as_index=False).head(1)
)


## give me only rows of the state column where the candidate is Donald trump
candidate_breakdown_republican[candidate_breakdown_republican.candidate == 'Donald Trump']['state']

1             Alabama
10            Arizona
14           Arkansas
18            Florida
23            Georgia
31            Indiana
49           Michigan
53        Mississippi
57           Missouri
61            Montana
64           Nebraska
68             Nevada
75      New Hampshire
80     North Carolina
94     South Carolina
99       South Dakota
103         Tennessee
116          Virginia
120        Washington
Name: state, dtype: object

In [13]:
votes


,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182
1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800
2,Alabama,AL,Baldwin,1003.0,Democrat,Bernie Sanders,2694,0.329
3,Alabama,AL,Baldwin,1003.0,Democrat,Hillary Clinton,5290,0.647
4,Alabama,AL,Barbour,1005.0,Democrat,Bernie Sanders,222,0.078
...,...,...,...,...,...,...,...,...
24606,Wyoming,WY,Teton-Sublette,95600028.0,Republican,Ted Cruz,0,0.000
24607,Wyoming,WY,Uinta-Lincoln,95600027.0,Republican,Donald Trump,0,0.000
24608,Wyoming,WY,Uinta-Lincoln,95600027.0,Republican,John Kasich,0,0.000
24609,Wyoming,WY,Uinta-Lincoln,95600027.0,Republican,Marco Rubio,0,0.000
